In [2]:
import urllib2, sys
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from IPython.core.display import HTML
import requests
import time
import datetime
import urlparse
import os

In [21]:
site = "https://www.hltv.org/stats/matches/heatmap/mapstatsid/52325/immortals-vs-cloud9?showKills=true&showDeaths=false&firstKillsOnly=false&allowEmpty=false&showKillDataset=true&showDeathDataset=true"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)
r = requests.get(site, headers=hdr)
tables = pd.read_html(r.text, header=0)

In [74]:
searching_data = soup.find_all( "div", class_ = "heatmap-holder")

In [86]:
def get_soup(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(url,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)
    return soup

s = get_soup('https://www.hltv.org/stats/matches/heatmap/mapstatsid/52325/immortals-vs-cloud9?showKills=true&showDeaths=false&firstKillsOnly=false&allowEmpty=false&showKillDataset=true&showDeathDataset=true')

divs = s.find_all('div', class_='heatmap heatmap-data')
outer = divs[0]['data-heatmap-config']
nested = divs[1]['data-heatmap-config']

In [88]:
outer

'{"renderAt":"uniqueHeatmapId1260517248","heatmapData":{"data":[{"x":286,"y":246,"value":1},{"x":489,"y":180,"value":1},{"x":360,"y":247,"value":1},{"x":493,"y":178,"value":1},{"x":584,"y":189,"value":1},{"x":454,"y":272,"value":1},{"x":453,"y":169,"value":1},{"x":459,"y":217,"value":1},{"x":265,"y":295,"value":1},{"x":261,"y":169,"value":1},{"x":269,"y":278,"value":1},{"x":411,"y":203,"value":1},{"x":226,"y":263,"value":1},{"x":366,"y":190,"value":1},{"x":278,"y":166,"value":1},{"x":436,"y":234,"value":1},{"x":406,"y":280,"value":1},{"x":270,"y":272,"value":1},{"x":342,"y":188,"value":1},{"x":338,"y":175,"value":1},{"x":198,"y":347,"value":1},{"x":276,"y":227,"value":1},{"x":417,"y":326,"value":1},{"x":381,"y":305,"value":1},{"x":367,"y":249,"value":1},{"x":228,"y":278,"value":1},{"x":271,"y":295,"value":1},{"x":514,"y":171,"value":1},{"x":505,"y":178,"value":1},{"x":592,"y":344,"value":1},{"x":584,"y":274,"value":1},{"x":532,"y":240,"value":1},{"x":296,"y":198,"value":1},{"x":311,"y"

In [215]:
def parse_stats_performance_page(url): 
    #site page example "https://www.hltv.org/stats/matches/performance/mapstatsid/52325/immortals-vs-cloud9"
    # THIS THE STATS/PERFORMANCE PAGE
    site = url
    hdr = {'User-Agent': 'Mozilla/5.0'}
    def site_soup(site):
        req = urllib2.Request(site,headers=hdr)
        page = urllib2.urlopen(req)
        soup = BeautifulSoup(page)
        return soup
    def get_performance_data(soup,site):
        r = requests.get(site, headers=hdr)
        tables = pd.read_html(r.text, header=0)
        total_team_kda = tables[0] 
        who_kill_who = tables[1] 
        first_kills = tables[2]  
        awp_kills = tables[3]   
        return {
            'total_team_kda': total_team_kda, # total kills deaths and assists of team
            'who_kill_who' : who_kill_who, # who killed who
            'first_kills' : first_kills, #first kill of the round
            'awp_kills' : awp_kills #awp kills
        } 
    
    soup = site_soup(site)
    try:                         #this section decides whether or not its a 3 map 2 map or 1map series
        match_3 = "No match_3"
        best_of_three_data = soup.find_all("div", class_ ="columns")[0]  # all dictated on whether or not
        links = []                                                       #this find_all finds the unique 'columns'   
        for link in best_of_three_data.find_all('a'):
            links.append(link.get("href"))
        site1 = "https://www.hltv.org" + str(links[1])
        site2 = "https://www.hltv.org" + str(links[2])
        try:
            site3 = "https://www.hltv.org" + str(links[3])
            soup = site_soup(site3) 
            match_3 = get_performance_data(soup,site3)
        finally:    
            soup = site_soup(site1) 
            match_1 = get_performance_data(soup,site1)
            soup = site_soup(site2) 
            match_2 = get_performance_data(soup,site2)
            return { 
                "match_1": match_1,
                "match_2": match_2,
                "match_3": match_3
            }
    except:
        match_1 = get_performance_data(soup, site)
        return{
            "match_1": match_1
        }

In [208]:
def parse_stats_page(url): 
    #site page example  "https://www.hltv.org/stats/matches/mapstatsid/52325/immortals-vs-cloud9"
    # THIS IS THE DETAILED STATS PAGE
    hdr = {'User-Agent': 'Mozilla/5.0'}   
    site = url
    def site_soup(site):
        req = urllib2.Request(site,headers=hdr)
        page = urllib2.urlopen(req)
        soup = BeautifulSoup(page)
        return soup

    
    def get_data_page(soup,site):
        match_time = soup.find_all("div", {"class":"small-text"})
        for item in match_time:
            match_time = item.text
        match_time = datetime.datetime.strptime(match_time, '%Y-%m-%d  %H:%MMap') #match date and time

        r = requests.get(site, headers=hdr)
        tables = pd.read_html(r.text, header=0)
        team_a_stats, team_b_stats = tables[0], tables[1]

        round_history_team = soup.find_all("div", class_ = "round-history-team-row") # winner of rounds and how rounds were won
        round_history_team_a = round_history_team[0].find_all("img")
        round_history_team_b = round_history_team[1].find_all("img")
        team_a_scores = []
        for scoreing in round_history_team_a:
             team_a_scores.append([ scoreing.get('title')])                    #rounds that team a won
        team_b_scores = []                                                         
        for scoreing in round_history_team_b:                              
             team_b_scores.append([ scoreing.get('title')])                    #rounds that team b won
        team_a_ending = []                                                          
        for ending in round_history_team_a:
            url = urlparse.urlparse(ending.get('src'))
            base = os.path.basename(url.path)                                  #how team a won the round
            team_a_ending.append([os.path.splitext(base)[0]])   
        team_b_ending = []
        for ending in round_history_team_b:
            url = urlparse.urlparse(ending.get('src'))
            base = os.path.basename(url.path)
            team_b_ending.append([os.path.splitext(base)[0]])                  #how team b won the round
        return {
            'match_time' : match_time, #match date and time
            'team_scores': [team_a_scores, team_b_scores], #rounds that team a won
            'team_endings': [team_a_ending, team_b_ending] #how the team won the round
           }
    
    soup = site_soup(site)
   
    try:             #this section decides whether or not its a 3 map 2 map or 1map series
        match_3 = "No match_3"
        best_of_three_data = soup.find_all("div", class_ ="columns")[0] # all dictated on whether or not
        links = []                                                    #this find_all finds the unique 'columns'   
        for link in best_of_three_data.find_all('a'):           
            links.append(link.get("href"))
        site1 = "https://www.hltv.org" + str(links[1])
        site2 = "https://www.hltv.org" + str(links[2])
        try:
            site3 = "https://www.hltv.org" + str(links[3])
            soup = site_soup(site3) 
            match_3 = get_data_page(soup,site3)
        finally:    
            soup = site_soup(site1) 
            match_1 = get_data_page(soup,site1)
            soup = site_soup(site2) 
            match_2 = get_data_page(soup,site2)
            return { 
                "match_1": match_1,
                "match_2": match_2,
                "match_3": match_3
            }
    except:
        match_1 = get_data_page(soup, site)
        return{
            "match_1": match_1
        }        

In [ ]:
site= "https://www.hltv.org/results?content=demo&team=7010&startDate=2016-09-01&endDate=2017-09-14"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [ ]:
matches = soup.find_all("div", class_='results-all')
results = matches[0].find_all("a", class_="a-reset")
urls1 = [result['href'] for result in results]

In [ ]:
site= "https://www.hltv.org/results?offset=100&startDate=2016-09-01&endDate=2017-09-14&content=demo&team=7010"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [ ]:
matches = soup.find_all("div", class_='results-all')
results = matches[0].find_all("a", class_="a-reset")
urls2 = [result['href'] for result in results]

In [ ]:
urls = urls1 + urls2

In [ ]:
def parse_match(site):
   #site page example https://www.hltv.org/matches/2315168/cloud9-vs-nrg-epicenter-2017-americas-closed-qualifier
    # THIS IS THE MATCH STATS PAGE
    hdr = {'User-Agent': 'Mozilla/5.0'}
    site = 'https://www.hltv.org' + site
    req = urllib2.Request(site,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)
    
   
    demo_url = 'https://www.hltv.org' + soup.find_all("a", class_="flexbox left-right-padding")[0]['href']
   
    try:
        vetos = soup.find_all("div", class_="standard-box veto-box")[1].find_all("div")[0].find_all("div")
        vetos = [veto.text for veto in vetos]
    except:
        vetos = None
        
        
    team_name = soup.find_all("div", class_="standard-box teamsBox")[0].find_all("div", class_ = "teamName")
    team_info = soup.find_all("div", class_="flexbox fix-half-width-margin maps")[0].find_all("div", class_ = "results")

    for name in team_name[0]:
        team_a_name = name
    for name in team_name[1]:
        team_b_name = name
    best_of = soup.find_all("div", class_ = "padding preformatted-text")
    for best in best_of:
        best_of = best.text
    if "Best of 3" in best_of:
        best_of = "Best of 3"
        map_name_one = soup.find_all("div", class_ = "mapname")[0]
        for name in map_name_one:
            map_name_one = name
        map_name_two = soup.find_all("div", class_ = "mapname")[1]
        for name in map_name_two:
            map_name_two = name
        map_name_three = soup.find_all("div", class_ = "mapname")[2]
        for name in map_name_three:
            map_name_three = name


        team_a_score_map_one = team_info[0].find_all("span")[0].text
        team_b_score_map_one = team_info[0].find_all("span")[2].text
        team_a_score_map_two = team_info[1].find_all("span")[0].text
        team_b_score_map_two = team_info[1].find_all("span")[2].text
        try:
            team_a_score_map_three = team_info[2].find_all("span")[0].text
            team_b_score_map_three = team_info[2].find_all("span")[2].text

        except: 
            team_b_score_map_three = 'NA'
            team_a_score_map_three = 'NA'
        
       
    elif "Best of 1" in best_of:
        best_of = "Best of 1"
        map_name_one = soup.find_all("div", class_ = "mapname")[0]
        for name in map_name_one:
            map_name_one = name
        team_a_score_map_one = team_info[0].find_all("span")[0].text
        team_b_score_map_one = team_info[0].find_all("span")[2].text
        map_name_two = team_a_score_map_two = team_b_score_map_two = map_name_three = team_a_score_map_three = team_b_score_map_three = 'NA'
       
    else:
        print("new type of match format")

    
    stats_url = 'https://www.hltv.org' + [a_element['href'] for a_element in soup.find_all("a") if a_element.text == "Detailed stats"][0]

    r = requests.get(site, headers=hdr)
    tables = pd.read_html(r.text, header=0)
    team_a, team_b = tables[0], tables[1]

    return {
        'team_a_b' : (team_a_name, team_b_name),
        'map_one': (map_name_one, team_a_score_map_one, team_b_score_map_one),
        'map_two': (map_name_two, team_a_score_map_two, team_b_score_map_two),
        'map_three': (map_name_three, team_a_score_map_three, team_b_score_map_three),
        'url': site,
        'vetos': vetos,
        'stats_url': stats_url,
        'teams': [team_a, team_b]
    }

In [ ]:
matches = []
for idx, url in enumerate(urls):
    matches.append(parse_match(url))
    time.sleep(5)
    print 'match {0} done'.format(idx)